# The Canonical Time-series Characteristics (catch22) transform

catch22\[1\] is a collection of 22 time series features extracted from the 7000+ present in the _hctsa_ \[2\]\[3\] toolbox.
A hierarchical clustering was performed on the correlation matrix of features that performed better than random chance to remove redundancy.
These clusters were sorted by balanced accuracy using a decision tree classifier and a single feature was selected from the 22 clusters formed, taking into account balanced accuracy results, computational efficiency and interpretability.

In this notebook, we will demonstrate how to use the catch22 transformer on the ItalyPowerDemand univariate and BasicMotions multivariate datasets. We also show catch22 used for classification with a random forest classifier.

#### References:

\[1\] Lubba, C. H., Sethi, S. S., Knaute, P., Schultz, S. R., Fulcher, B. D., & Jones, N. S. (2019). catch22: CAnonical Time-series CHaracteristics. Data Mining and Knowledge Discovery, 33(6), 1821-1852.

\[2\] Fulcher, B. D., & Jones, N. S. (2017). hctsa: A computational framework for automated time-series phenotyping using massive feature extraction. Cell systems, 5(5), 527-531.

\[3\] Fulcher, B. D., Little, M. A., & Jones, N. S. (2013). Highly comparative time-series analysis: the empirical structure of time series and their methods. Journal of the Royal Society Interface, 10(83), 20130048.

In [1]:
%pip install ..[dev]

Looking in indexes: https://srv01af2.corp.trumpf.com/artifactory/api/pypi/pypi-remote/simple
Processing c:\kod\sktime
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for sktime: filename=sktime-0.27.0-py3-none-any.whl size=27742555 sha256=bb06b61f9b34bd147ca38de38cbfcd5dfdf2f05a03af1e408e12cf5c8ad14e66
  Stored in directory: C:\Users\nowakju\AppData\Local\Temp\pip-ephem-wheel-cache-looxlmvx\wheels\dd\5e\5d\e2679483802bc8e7237c5aae9f7a2da9ccdaaeaeb066d861f5
Successfully built sktime
  Attempting uninstall: sktime
    Found existing installation: sktime 0.27.0
    Uninstalling sktime-0.27.0:

## 1. Imports

In [2]:
from sklearn import metrics

from sktime.classification.feature_based import Catch22Classifier
from sktime.datasets import load_basic_motions, load_italy_power_demand
from sktime.transformations.panel.catch22 import Catch22

## 2. Load data

In [3]:
IPD_X_train, IPD_y_train = load_italy_power_demand(split="train", return_X_y=True)
IPD_X_test, IPD_y_test = load_italy_power_demand(split="test", return_X_y=True)
IPD_X_test = IPD_X_test[:50]
IPD_y_test = IPD_y_test[:50]

print(IPD_X_train.shape, IPD_y_train.shape, IPD_X_test.shape, IPD_y_test.shape)

BM_X_train, BM_y_train = load_basic_motions(split="train", return_X_y=True)
BM_X_test, BM_y_test = load_basic_motions(split="test", return_X_y=True)

print(BM_X_train.shape, BM_y_train.shape, BM_X_test.shape, BM_y_test.shape)

(67, 1) (67,) (50, 1) (50,)
(40, 6) (40,) (40, 6) (40,)


## 3. catch22 transform

### Univariate

The catch22 features are provided in the form of a transformer, `Catch22`.
From this the transformed data can be used for a variety of time series analysis tasks.
Please note, that Catch22 doesn't take labels (`y`) into consideration in the `fit(x, y=None)` method.

In [4]:
c22_uv = Catch22()
c22_uv.fit(IPD_X_train, IPD_y_train)

Catch22()

In [5]:
transformed_data_uv = c22_uv.transform(IPD_X_train)
print(transformed_data_uv.head())

         0         1     2         3         4    5     6         7   \
0  1.158630 -0.217227   8.0  0.291667 -0.625000  3.0   6.0  0.468052   
1  0.918162 -0.214762  15.0  0.208333 -0.666667  4.0   8.0  0.702775   
2 -0.273180 -0.085856   4.0  0.875000  0.250000  2.0   5.0  0.310567   
3  0.048411 -0.450080  13.0  0.166667 -0.625000  4.0  10.0  0.804047   
4  0.426379  0.572566  16.0  0.291667 -0.666667  4.0   7.0  0.675485   

         8         9   ...   12        13   14        15        16        17  \
0  0.589049  0.836755  ...  3.0  1.000000  5.0  1.778748  0.750000  0.240598   
1  0.196350  0.666160  ...  4.0  0.869565  5.0  1.730238  0.500000  0.388217   
2  0.589049  0.865073  ...  2.0  0.913043  5.0  1.836012  0.666667  0.089104   
3  0.196350  0.648309  ...  4.0  0.869565  6.0  1.605420  0.666667  0.332436   
4  0.196350  0.657946  ...  4.0  0.913043  6.0  1.730238  0.500000  0.318405   

   18  19        20   21  
0 NaN NaN  0.040000  0.0  
1 NaN NaN  0.111111  0.0  
2 NaN

### Multivariate

Transformation of multivariate data is supported by `Catch22`.
The default procedure will concatenate each column prior to transformation.
We can also set specific column names, e.g., `"short_str_feat"` which will show short name of the feauture in the column name.

In [6]:
c22_mv = Catch22(col_names="short_str_feat")
c22_mv.fit(BM_X_train, BM_y_train)

Catch22(col_names='short_str_feat')

In [7]:
transformed_data_mv = c22_mv.transform(BM_X_train)
print(transformed_data_mv.head())

   dim_0__mode_5  dim_0__mode_10  dim_0__stretch_decreasing  \
0      -0.140988       -0.268073                        6.0   
1      -0.387256       -0.126246                        6.0   
2       0.028412       -0.224988                        9.0   
3      -0.147338       -0.199523                        8.0   
4      -0.217645       -0.252015                        7.0   

   dim_0__outlier_timing_pos  dim_0__outlier_timing_neg  dim_0__acf_timescale  \
0                     -0.890                      0.160                   2.0   
1                     -0.920                     -0.600                   2.0   
2                     -0.335                     -0.045                   1.0   
3                     -0.540                      0.180                   1.0   
4                     -0.130                      0.020                   1.0   

   dim_0__acf_first_min  dim_0__centroid_freq  dim_0__low_freq_power  \
0                   3.0              0.042638               0.

## 4. catch22 Forest Classifier

For classification tasks the default classifier to use with the catch22 features is random forest classifier.
An implementation making use of the `RandomForestClassifier` from sklearn built on catch22 features is provided in the form on the `Catch22Classifier` for ease of use.

In [ ]:
c22f = Catch22Classifier(random_state=0)
c22f.fit(IPD_X_train, IPD_y_train)

In [ ]:
c22f_preds = c22f.predict(IPD_X_test)
print("C22F Accuracy: " + str(metrics.accuracy_score(IPD_y_test, c22f_preds)))